In [19]:
import pandas as pd
import csv
import os
import json
import numpy as np
from citipy import citipy
import requests
import matplotlib.pyplot as plt


In [20]:
os.chdir("../python-api-challenge")

In [21]:
from config import api_key

ModuleNotFoundError: No module named 'config'

In [ ]:
from datetime import date

today = date.today()

d1 = today.strftime("%m/%d/%Y")
print("Today's Date:", d1)

In [ ]:
# create list of random latitudes evenly distributed across globe

lat = np.random.uniform(low=-90.000, high=90.000, size=2000)

In [ ]:
# create list of random longitudes evenly distributed across globe

long = np.random.uniform(low=-180.000, high=180.000, size=2000)

In [ ]:
# combine lat and long

latlong = zip(lat, long)

In [ ]:
# create list of latitude and longitude combinations 

coordinates=list(latlong)

In [ ]:
latlong

In [ ]:
coordinates

In [ ]:
# create empty lists to be populated in loops

cities=[]

city_name=[]
country=[]
city_id=[]
latitude=[]
longitude=[]
max_temp=[]
humidity=[]
wind_speed=[]
cloudiness=[]

In [ ]:
# populate cities list

for x in coordinates:
    city = citipy.nearest_city(x[0],x[1]).city_name
    if city not in cities:
        cities.append(city)
  

In [ ]:
# create vairable for base url to be used when making api calls

url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial"
# city = []



In [ ]:
# loop through city names and create unique api calls for each city
# populate lists for desired fields from returned json
# skip cities that so not result in a successful api call

for city in cities:

    query_url = url + "&appid=" + api_key + "&q=" + city
    weather_response = requests.get(query_url).json()

    try:
        city_name.append(weather_response["name"])
        country.append(weather_response["sys"]["country"])
        city_id.append(weather_response["id"])
        latitude.append(weather_response["coord"]["lat"])
        longitude.append(weather_response["coord"]["lon"])
        max_temp.append(weather_response["main"]["temp_max"])
        humidity.append(weather_response["main"]["humidity"])
        wind_speed.append(weather_response["wind"]["speed"])
        cloudiness.append(weather_response["clouds"]["all"])
        print(city, "success")
    except:
        print(city, "CITY NOT FOUND!")
    pass


In [ ]:
# create pandas dataframe from lists populated by series of api calls

results_df=pd.DataFrame({"City":pd.Series(city_name), 
                         "Country":pd.Series(country),
                         "ID":pd.Series(city_id),
                         "Latitude":pd.Series(latitude), 
                         "Longitude":pd.Series(longitude), 
                         "Max Temp":pd.Series(max_temp), 
                         "Humidity":pd.Series(humidity), 
                         "Wind Speed":pd.Series(wind_speed),
                         "Cloudiness":pd.Series(cloudiness)
                        })

results_df

In [ ]:
# export dataframe to a csv file

results_df.to_csv("WeatherPy/results.csv")

In [ ]:
plt.scatter(results_df["Latitude"], results_df["Max Temp"])
plt.title(f"°Latitude vs. Temperature {d1}")
plt.ylabel("Temperature (°F)")
plt.xlabel("°Latitude")
plt.grid(True)

plt.savefig("figures/max_temp.png")

<h1> °Latitude vs. Temperature <h1>
<h3>Not surprisingly, the highest temeratures are nearest the equator.  Since nearly 70% of the planets land is in the northern hemisphere, it makes sense that there are significantly fewer data points in the southern hemisphere.  The seasonal tilt of the Earths axis likely is the cause for relatively higher temps in the south compared to the north.<h3>

In [ ]:
plt.scatter(results_df["Latitude"], results_df["Humidity"])
plt.title(f"°Latitude vs. Humidity {d1}")
plt.ylabel("% Humidity")
plt.xlabel("°Latitude")
plt.grid(True)

plt.savefig("figures/humidity.png")

<h1> °Latitude vs. Humidity <h1>
<h3>Beyond a slight concentration of zero humidity locations, there appears to be little correlation between latitude and humidity levels.<h3>

In [ ]:
plt.scatter(results_df["Latitude"], results_df["Wind Speed"])
plt.title(f"°Latitude vs. Wind Speed {d1}")
plt.ylabel("Wind Speed")
plt.xlabel("°Latitude")
plt.grid(True)

plt.savefig("figures/wind_speed.png")

<h1> °Latitude vs. Wind Speed <h1>
<h3>Althought the extreme northern latitudes do indicate an increased possibility of some of the higher wind speeds, most of the data indicates little correlation. It is important to note that we lack data points from the extreme southern latitudes that may better bring this significance into focus<h3>

In [ ]:
plt.scatter(results_df["Latitude"], results_df["Cloudiness"])
plt.title(f"°Latitude vs. Cloudiness {d1}")
plt.ylabel("Cloudiness")
plt.xlabel("°Latitude")
plt.grid(True)

plt.savefig("figures/cloudiness.png")

<h1> °Latitude vs. Cloudiness <h1>
<h3>There appears to be no significant direct correlation between latitude and cloudiness.s<h3>

In [ ]:
# create dataframe of cities in the northern hemisphere

north_hem = results_df.Latitude >= 0
north_df = results_df[north_hem]
north_df
north_df.to_csv("WeatherPy/north_df.csv")

In [ ]:
# create dataframe of cities in the southern hemisphere

south_hem = results_df.Latitude < 0
south_df = results_df[south_hem]
south_df
south_df.to_csv("WeatherPy/south_df.csv")

In [ ]:
from scipy import stats

def stats_linregress(x,y):
    res = stats.linregress(x,y)
    plt.plot(x, y, "+", label="Cities")
    plt.plot(x, res.intercept + res.slope* x, "r", label="Linear Regression", color="magenta")
    plt.legend()
    plt.xlabel("°Latitude")
    print(f"R-squared: {res.rvalue**2:.6f}")
#     plt.show()

# res = stats.linregress(south_df["Latitude"], south_df["Wind Speed"])

In [ ]:
north_temp = stats_linregress(north_df["Latitude"], north_df["Max Temp"])
plt.title(f"°Latitude vs. Max Temp - Northern Hemisphere {d1}")
plt.ylabel("Max Temp")
plt.savefig("figures/north_temp.png")
# plt.show()


In [ ]:
south_temp = stats_linregress(south_df["Latitude"], south_df["Max Temp"])
plt.title(f"°Latitude vs. Max Temp - Southern Hemisphere {d1}")
plt.ylabel("Max Temp")
# plt.show()
plt.savefig("figures/south_temp.png")

<h2>°Latitude vs. Max Temp<h2>
<img src="../figures/north_temp.png" height:50px/>
<img src="../figures/south_temp.png" height:50px/>
<h3>There is a much tighter coorelation to betweem max temp and latitude in the northern hemisphere.  This is likely due to the amount of data points and the relative diversity of the land dispersion in the southern hemisphere.<h3>


In [ ]:
north_humidity = stats_linregress(north_df["Latitude"], north_df["Humidity"])
plt.title(f"°Latitude vs. Humidity - Northern Hemisphere {d1}")
plt.ylabel("% Humidity")
plt.savefig("figures/north_humidity.png")
# plt.show()

In [ ]:
south_humidity = stats_linregress(south_df["Latitude"], south_df["Humidity"])
plt.title(f"°Latitude vs. Humidity - Southern Hemisphere {d1}")
plt.ylabel("% Humidity")
plt.savefig("figures/south_humidity.png")
# plt.show()


<h2>°Latitude vs. Humidity<h2>
<img src="../figures/north_humidity.png" height:50px/>
<img src="../figures/south_humidity.png" height:50px/>
<h3>Although there is a a loose coorelation between latitude and humidity, and that the coorelation is similiar between the hemispheres, nearly all levels of humidity are present at all latitudes.<h3>

In [ ]:
north_wind = stats_linregress(north_df["Latitude"], north_df["Wind Speed"])
plt.title(f"°Latitude vs. Wind Speed - Northern Hemisphere {d1}")
plt.ylabel("Wind Speed")
plt.savefig("figures/north_wind.png")
# plt.show()

In [ ]:
south_wind = stats_linregress(south_df["Latitude"], south_df["Wind Speed"])
plt.title(f"°Latitude vs. Wind Speed - Southern Hemisphere {d1}")
plt.ylabel("Wind Speed")
plt.savefig("figures/south_wind.png")
# plt.show()


<h2>°Latitude vs. Wind Speed<h2>
<img src="../figures/north_wind.png" height:50px/>
<img src="../figures/south_wind.png" height:50px/>
<h3>There is not a notable coorelation between latitude and wind speed.<h3>

In [ ]:
north_cloudiness = stats_linregress(north_df["Latitude"], north_df["Cloudiness"])
plt.title(f"°Latitude vs. Cloudiness - Northern Hemisphere {d1}")
plt.ylabel("Cloudiness")
plt.savefig("figures/north_cloudiness.png")
# plt.show()

In [ ]:
south_cloudiness = stats_linregress(south_df["Latitude"], south_df["Cloudiness"])
plt.title(f"°Latitude vs. Cloudiness - Southern Hemisphere {d1}")
plt.ylabel("Cloudiness")
plt.savefig("figures/south_cloudiness.png")
# plt.show()


<h2>°Latitude vs. Cloudiness<h2>
<img src="../figures/north_cloudiness.png" height:50px/>
<img src="../figures/south_cloudiness.png" height:50px/>
<h3>There is not a notable coorelation between latitude and cloudiness.<h3>